# Clustering Crypto

In [30]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [31]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

df_url = requests.get(url).json()

In [32]:
df_url_1 = pd.DataFrame(df_url["Data"]).T
df_url_1.head(5)

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,BlockTime,AssetLaunchDate,MaxSupply,MktCapPenalty,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,IsUsedInDefi
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,0,2017-07-01,300,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [34]:
# Alternatively, use the provided csv file:
from pathlib import Path
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
df = pd.read_csv(file_path, index_col="Unnamed: 0")
print(df.shape)
df.head()


(1252, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
df.drop(columns=["Unnamed: 0"], inplace = True)
df


In [35]:
# Keep only cryptocurrencies that are trading
df = df[df["IsTrading"] == True]
#df = df.loc[df["IsTrading"] == True]
print(df.shape)
df.head(5)

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [36]:
# Keep only cryptocurrencies with a working algorithm
df = df.loc[df["Algorithm"] != "N/A"]
print(df.shape)
df.head(5)

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [37]:
# Remove the "IsTrading" column
df.drop(columns=["IsTrading"], axis=1,inplace=True)

In [38]:
print(df.shape)
df.head(5)

(1144, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [39]:
# Remove rows with at least 1 null value
df =  df.dropna(axis=0, how="any")

In [40]:
print(df.shape)
df.head(5)

(685, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [41]:
# Remove rows with cryptocurrencies having no coins mined
df = df.loc[df["TotalCoinsMined"] > 0]
print(df.shape)
df.head(5)

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [42]:
# Drop rows where there are 'N/A' text value
df =  df[df.iloc[:] != 'N/A'].dropna()

In [43]:
print(df.shape)
df.head(5)

(532, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [44]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coins_name = pd.DataFrame(df["CoinName"], index=df.index)
print(coins_name.shape)
coins_name.head(5)

(532, 1)


,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [45]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
df = df.drop(columns=["CoinName"], axis=1)

In [46]:
print(df.shape)
df.head(5)

(532, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [47]:
# Create dummy variables for text features
X = pd.get_dummies(df, columns=["Algorithm","ProofType"])
print(X.shape)
X.head(5)

(532, 98)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Standardize data
X = StandardScaler().fit_transform(X)
print(X[0:5])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Reducing Dimensions Using PCA

In [49]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X)
crypto_pca

array([[-0.33490337,  0.95234178, -0.56214846],
       [-0.31821495,  0.9520519 , -0.56250599],
       [ 2.33324771,  1.31448497, -0.65817077],
       ...,
       [ 0.3244816 , -2.3350281 ,  0.38081813],
       [-0.15209998, -1.93949693,  0.44377037],
       [-0.31108508,  1.07449544, -0.23484157]])

In [50]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(data=crypto_pca, columns=["PC1", "PC2", "PC3"], index=df.index)
print(df_crypto_pca.shape)
df_crypto_pca.head()

(532, 3)


,PC1,PC2,PC3
42,-0.334903,0.952342,-0.562148
404,-0.318215,0.952052,-0.562506
1337,2.333248,1.314485,-0.658171
BTC,-0.144902,-1.305352,0.149104
ETH,-0.156695,-1.983695,0.358555


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [51]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [52]:
# Initialize the K-Means model
model= KMeans(n_clusters= 4, random_state=0)
# Fit the model
model.fit(df_crypto_pca)
# Predict clusters
prediction = model.predict(df_crypto_pca)

#---------------------------------------

# Create a new DataFrame including predicted clusters and cryptocurrencies features
cluster_df = pd.concat([df,df_crypto_pca], axis=1)
cluster_df['CoinName'] = coins_name['CoinName']
cluster_df["class"] = model.labels_
print(cluster_df.shape)
cluster_df.head()


(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.334903,0.952342,-0.562148,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.318215,0.952052,-0.562506,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.333248,1.314485,-0.658171,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144902,-1.305352,0.149104,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.156695,-1.983695,0.358555,Ethereum,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [53]:
# Create a 3D-Scatter with the PCA data and the cluster
fig = px.scatter_3d(
    cluster_df,
    x="PC1",
    y="PC2",
    z="PC3",
    color="class",
    symbol="class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [54]:
# Table with tradable cryptos
cluster_df[["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"]].hvplot.table()

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [55]:
# Print the total number of tradable cryptocurrencies

print(f"The total number of tradable cryptocurrencies is",{len(cluster_df)})

The total number of tradable cryptocurrencies is {532}


#### Scatter Plot with Tradable Cryptocurrencies

In [56]:
# Scale data to create the scatter plot
mm_scal = MinMaxScaler()
plot_data = mm_scal.fit_transform(cluster_df[['TotalCoinsMined','TotalCoinSupply']])
plot_df = pd.DataFrame(plot_data, columns=['TotalCoinsMined','TotalCoinSupply'], index=cluster_df.index)
plot_df['CoinName'] = cluster_df['CoinName']
plot_df['class'] = cluster_df['class']
plot_df.head(5)


,TotalCoinsMined,TotalCoinSupply,CoinName,class
42,0.000000,4.200000e-11,42 Coin,0
404,0.001066,5.320000e-04,404Coin,0
1337,0.029576,3.141593e-01,EliteCoin,0
BTC,0.000018,2.100000e-05,Bitcoin,1
ETH,0.000109,0.000000e+00,Ethereum,1


In [57]:
 plot_df.hvplot.scatter(x="TotalCoinsMined",y="TotalCoinSupply",hover_color=["CoinName"], by="class")



:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply)